## TEST NOTEBOOK

In [ ]:
import wrds
import pandas as pd
from datetime import datetime
from pathlib import Path


import numpy as np
import pandas as pd
import wrds
from dateutil.relativedelta import relativedelta

from settings import config

DATA_DIR = Path(config("DATA_DIR"))
WRDS_USERNAME = config("WRDS_USERNAME")
START_DATE = config("START_DATE")
END_DATE = config("END_DATE")

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

TEMP_DATA_DIR = Path("/tmp/ravenpack_pull")
TEMP_DATA_DIR.mkdir(exist_ok=True)


def pull_year(year):
    """Pull one year in its own WRDS connection (thread-safe)."""
    out_path = TEMP_DATA_DIR / f"ravenpack_{year}.parquet"
    if out_path.exists():
        print(f"  {year}: already downloaded, skipping")
        return
    query = f"""
    WITH SingleStockStories AS (
        SELECT rp_story_id
        FROM ravenpack_dj.rpa_djpr_equities_{year}
        WHERE country_code = 'US' AND relevance = 100
        GROUP BY rp_story_id
        HAVING COUNT(DISTINCT rp_entity_id) = 1
    )
    SELECT
        t.rpa_date_utc, t.rpa_time_utc, t.timestamp_utc,
        t.rp_story_id, t.rp_entity_id, t.entity_name,
        t.headline, t.news_type, t.relevance,
        t.event_sentiment_score
    FROM ravenpack_dj.rpa_djpr_equities_{year} t
    JOIN SingleStockStories s ON t.rp_story_id = s.rp_story_id
    WHERE t.country_code = 'US' AND t.relevance = 100
    ORDER BY t.timestamp_utc ASC
    """
    db = wrds.Connection(wrds_username=WRDS_USERNAME)
    df = db.raw_sql(query, date_cols=["rpa_date_utc", "rpa_time_utc", "timestamp_utc"])
    db.close()
    print(f"  {year}: {len(df):,} rows")
    df.to_parquet(out_path)
    del df


years = range(2000, 2020)
with ThreadPoolExecutor(max_workers=3) as executor:
    futures = {executor.submit(pull_year, y): y for y in years}
    for future in as_completed(futures):
        year = futures[future]
        try:
            future.result()
        except Exception as e:
            print(f"Error pulling year {year}: {e}")

print("Done. Files saved to", TEMP_DATA_DIR)


Loading library list...
Loading library list...
Loading library list...
Loading library list...
Loading library list...
Done
Done
Done
Done
Done


/opt/homebrew/Caskroom/miniconda/base/envs/p17_chen_kelly_xiu_2022/lib/python3.12/site-packages/wrds/sql.py:603: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in df:


  2000: 193,985 rows
Loading library list...
Done


/opt/homebrew/Caskroom/miniconda/base/envs/p17_chen_kelly_xiu_2022/lib/python3.12/site-packages/wrds/sql.py:603: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in df:


  2002: 419,372 rows
Loading library list...
Done


...

  2001: 458,162 rows
Loading library list...
Done


/opt/homebrew/Caskroom/miniconda/base/envs/p17_chen_kelly_xiu_2022/lib/python3.12/site-packages/wrds/sql.py:603: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in df:
/opt/homebrew/Caskroom/miniconda/base/envs/p17_chen_kelly_xiu_2022/lib/python3.12/site-packages/wrds/sql.py:603: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in df:


  2003: 445,716 rows
Loading library list...


/opt/homebrew/Caskroom/miniconda/base/envs/p17_chen_kelly_xiu_2022/lib/python3.12/site-packages/wrds/sql.py:603: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in df:


Done
  2004: 959,079 rows
Loading library list...
Done


In [ ]:
with wrds.Connection(wrds_username=WRDS_USERNAME) as db:
    try:
        # List all available libraries (schemas)
        libraries = db.list_libraries()
        # print(libraries)

        # Filter for RavenPack specifically
        ravenpack_libs = [lib for lib in sorted(libraries) if "raven" in lib.lower()]
        print("RavenPack libraries:", ravenpack_libs)

        for lib in ravenpack_libs:
            print(f"Tables in {lib} library:")
            tables = db.list_tables(library=lib)
            print(tables)
    except Exception as e:
        print("An error occurred while connecting to WRDS or fetching data:", e)

## RavenPack Libraries on WRDS

### 1. `ravenpack_common`
Reference/mapping tables:

| Table | Description |
|-------|-------------|
| `common_chars` | Common characteristics |
| `rpa_company_mappings` | Company ID mappings |
| `rpa_entity_mappings` | Entity ID mappings |
| `rpa_source_list` | News source list |
| `rpa_taxonomy` | Event taxonomy/categories |
| `wrds_rpa_all_mappings` | WRDS combined mappings |
| `wrds_rpa_company_mappings` | WRDS company mappings |
| `wrds_rpa_company_names` | Company names |
| `wrds_rpa_entity_mappings` | WRDS entity mappings |
| `wrds_rpa_source_list` | WRDS source list |

### 2. `ravenpack_dj` (Dow Jones Edition)

| Table Type | Years | Description |
|------------|-------|-------------|
| `djpr_chars` | - | Characteristics/metadata |
| `rpa_djpr_equities_YYYY` | 2000-2025 | Equity news sentiment (26 tables) |
| `rpa_djpr_global_macro_YYYY` | 2000-2025 | Macro news sentiment (26 tables) |

### 3. `ravenpack_trial`

| Table | Description |
|-------|-------------|
| `chars` | Characteristics |
| `rpa_entity_mappings` | Entity mappings |
| `rpa_full_equities` | Full equities data (single table) |
| `rpa_full_global_macro` | Full macro data (single table) |
| `rpa_source_list` | Source list |
| `rpa_taxonomy` | Event taxonomy |
| `wrds_rpa_all_mappings` | Combined mappings |
| `wrds_rpa_company_names` | Company names |

**Note:** The trial library has consolidated tables (`rpa_full_*`) while the DJ edition splits data by year.

In [ ]:
# with wrds.Connection(wrds_username=WRDS_USERNAME) as db:
#     # db.describe_table(library="ravenpack_trial", table="rpa_full_equities")
#     # df = db.get_table(library="ravenpack_trial", table="rpa_full_equities")
#     # print(df.head())
#     query = """
#     SELECT * FROM ravenpack_dj.rpa_djpr_equities_2000
#     WHERE country_code = 'US'
# """
#     df = db.raw_sql(query)


# filters = f"""country_code = 'US'
#     AND timestamp_utc >= '{START_DATE}'
#     AND timestamp_utc <= '{END_DATE}'"""
query = """
WITH SingleStockStories AS (
    SELECT
        rp_story_id
    FROM ravenpack_dj.rpa_djpr_equities_{YEAR}
    WHERE
        country_code = 'US'
        AND relevance = 100
    GROUP BY rp_story_id
    HAVING COUNT(DISTINCT rp_entity_id) = 1
)

SELECT
    *
FROM ravenpack_dj.rpa_djpr_equities_{YEAR} t
JOIN SingleStockStories s
    ON t.rp_story_id = s.rp_story_id
WHERE
    t.country_code = 'US'
    AND t.relevance = 100
"""

with wrds.Connection(wrds_username=WRDS_USERNAME) as db:
    df = db.raw_sql(query.format(YEAR=2000))


# df.head()

In [ ]:
"""
Index(['rpa_date_utc', 'rpa_time_utc', 'timestamp_utc', 'rp_story_id',
       'rp_entity_id', 'entity_type', 'entity_name', 'country_code',
       'relevance', 'event_sentiment_score', 'event_relevance',
       'event_similarity_key', 'event_similarity_days', 'topic', 'group',
       'type', 'sub_type', 'property', 'fact_level', 'rp_position_id',
       'position_name', 'evaluation_method', 'maturity', 'earnings_type',
       'event_start_date_utc', 'event_end_date_utc', 'reporting_period',
       'reporting_start_date_utc', 'reporting_end_date_utc', 'related_entity',
       'relationship', 'category', 'event_text', 'news_type', 'rp_source_id',
       'source_name', 'css', 'nip', 'peq', 'bee', 'bmq', 'bam', 'bca', 'ber',
       'anl_chg', 'mcq', 'rp_story_event_index', 'rp_story_event_count',
       'product_key', 'provider_id', 'provider_story_id', 'headline'],
      dtype='object')
"""
with wrds.Connection(wrds_username=WRDS_USERNAME) as db:
    query = """
    WITH SingleStockStories AS (
        SELECT
            rp_story_id
        FROM ravenpack_dj.rpa_djpr_equities_2000
        WHERE
            country_code = 'US'
            AND relevance = 100
        GROUP BY rp_story_id
        HAVING COUNT(DISTINCT rp_entity_id) = 1
    )
    SELECT 
        t.rpa_date_utc,
        t.rpa_time_utc,
        t.timestamp_utc,
        t.rp_story_id,
        t.rp_entity_id,
        t.entity_name,
        t.headline,
        t.news_type,
        t.relevance
      FROM ravenpack_dj.rpa_djpr_equities_2000 t
    JOIN SingleStockStories s
        ON t.rp_story_id = s.rp_story_id
    WHERE 
        t.country_code = 'US'
        AND t.relevance = 100
    ORDER BY t.timestamp_utc ASC
"""
    df2 = db.raw_sql(query, date_cols=["rpa_date_utc", "rpa_time_utc", "timestamp_utc"])


df2.head()

In [ ]:
with wrds.Connection(wrds_username=WRDS_USERNAME) as db:
    mappings = db.get_table(
        library="ravenpack_common", table="wrds_rpa_company_mappings"
    )

In [ ]:
mappings

: 

In [ ]:
df_all

,rpa_date_utc,rpa_time_utc,timestamp_utc,rp_story_id,rp_entity_id,entity_name,headline,news_type,relevance,event_sentiment_score
0,2000-01-03,2026-02-10 07:00:00,2000-01-03 07:00:00.000,DFC2614452E1D8749F6F2C2886A5D2C0,F70C58,IEC Electronics Corp.,IEC Electronics Posts Wider Loss on Costs To...,FULL-ARTICLE,100.0,-0.65
1,2000-01-03,2026-02-10 07:00:00,2000-01-03 07:00:00.000,6D3B043E7E2F449DEB35042B68F0C971,C8BF25,U S West Inc.,U S West Sells 65% Of Stake in Ex-Partner Gl...,FULL-ARTICLE,100.0,0.22
2,2000-01-03,2026-02-10 07:00:00,2000-01-03 07:00:00.000,6D3B043E7E2F449DEB35042B68F0C971,C8BF25,U S West Inc.,U S West Sells 65% Of Stake in Ex-Partner Gl...,FULL-ARTICLE,100.0,0.18
3,2000-01-03,2026-02-10 07:00:00,2000-01-03 07:00:00.000,97AEB7DE31006C706113F21B1AADA4F6,FD615E,Bethlehem Corp.,Business Brief -- BETHLEHEM CORP.: Zizza Resi...,FULL-ARTICLE,100.0,0.53
4,2000-01-03,2026-02-10 07:00:00,2000-01-03 07:00:00.000,5AFCFAAEE719446723D653FDDF8EE6F4,246C0A,CHS Inc.,Cenex to Acquire Sparta Foods,FULL-ARTICLE,100.0,0.61
...,...,...,...,...,...,...,...,...,...,...
16364843,2019-12-31,2026-02-10 23:03:27,2019-12-31 23:03:27.145,D9BADBD5432F021CD1B2D75B850CDB15,F5A93A,Salem Media Group Inc.,"CEO Atsinger III Buys 20,120 Of Salem Media Gr...",TABULAR-MATERIAL,100.0,0.52
16364844,2019-12-31,2026-02-10 23:27:58,2019-12-31 23:27:58.050,58F0B5CB1BF0700939432DB8C342862A,58EA22,Cue Biopharma Inc.,"Dir Marlett Sells 24,700 Of Cue BioPharma Inc ...",TABULAR-MATERIAL,100.0,-0.49
16364845,2019-12-31,2026-02-10 23:29:26,2019-12-31 23:29:26.034,EF7735E34C2BB5C7C5FD4CAAEF1FE81E,092C47,Lumentum Holdings Inc.,"VP Hamel Sells 3,113 Of Lumentum Holdings Inc ...",TABULAR-MATERIAL,100.0,-0.49
16364846,2019-12-31,2026-02-10 23:53:39,2019-12-31 23:53:39.057,20DE5CF71960B6E4B365BA6C6FA9D91B,15EC17,Amarillo Biosciences Inc.,"Chmn Chen Buys 7,000 Of Amarillo Biosciences I...",TABULAR-MATERIAL,100.0,0.52
